In [15]:
import tensorflow as tf
import keras
from tensorflow.keras.layers import Layer


In [22]:
def inception_module(x,
                     filter_1x1,
                     filters_3x3_reduce,
                     filters_3x3,
                     filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj,
                     name = None):
    conv_1x1 = tf.keras.layers.Conv2D(filter_1x1, (1, 1), padding='same', activation='relu')(x)
    conv_3x3 = tf.keras.layers.Conv2D(filters_3x3_reduce,(1,1), padding='same', activation='relu')(x)
    conv_3x3 = tf.keras.layers.Conv2D(filters_3x3,(3,3),padding='same',activation='relu')(conv_3x3)
    conv_5x5 = tf.keras.layers.Conv2D(filters_5x5_reduce,(1,1),padding='same',activation='relu')(x)
    conv_5x5 = tf.keras.layers.Conv2D(filters_5x5,(5,5),padding='same',activation='relu')(conv_5x5)
    pool_proj = tf.keras.layers.MaxPooling2D((3,3),strides=(1,1),padding='same')(x)
    pool_proj = tf.keras.layers.Conv2D(filters_pool_proj,(1,1),padding='same',activation='relu')(pool_proj)
    output = tf.keras.layers.concatenate([conv_1x1,conv_3x3,conv_5x5,pool_proj],axis=3,name=name)

    return output


In [19]:
import tensorflow as tf

In [52]:
from tensorflow.keras.layers import AveragePooling2D, Conv2D, Flatten, Dense, Dropout

input_layer = tf.keras.layers.Input(shape=(224, 224, 3))


x = tf.keras.layers.Conv2D(64, (7,7), strides=(2,2), padding='same', activation='relu', name='conv_1_7x7_2')(input_layer)
x = tf.keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='same', name='max_pool_1_3x3_2')(x)
x = tf.keras.layers.Conv2D(192, (3,3), strides=(1,1), padding='same', activation='relu', name='conv_2b_3x3_1')(x)
x = tf.keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='same', name='max_pool_2_3x3_2')(x)


x = inception_module(x,64,96,128,16,32,32,name='inception_3a')

x = inception_module(x,128,128,192,32,96,64,name='inception_3b')

x = tf.keras.layers.MaxPooling2D((3,3),strides=(2,2),padding="same",name='max_pool_3_3x3_2')(x)

x = inception_module(x,192,96,208,16,48,64,name='inception_4a')

x1 = AveragePooling2D((5, 5), strides=3)(x)
x1 = Conv2D(128, (1, 1), padding='same', activation='relu')(x1)
x1 = Flatten()(x1)
x1 = Dense(1024, activation='relu')(x1)
x1 = Dropout(0.7)(x1)
x1 = Dense(10, activation='softmax', name='auxilliary_output_1')(x1)

x = inception_module(x,160,112,224,24,64,64,name='inception_4b')

x = inception_module(x,128,128,256,24,64,64,name='inception_4c')

x = inception_module(x,112,144,288,32,64,64,name='inception_4d')

x2 = AveragePooling2D((5, 5), strides=3)(x)
x2 = Conv2D(128, (1, 1), padding='same', activation='relu')(x2)
x2 = Flatten()(x2)
x2 = Dense(1024, activation='relu')(x2)
x2 = Dropout(0.7)(x2)
x2 = Dense(10, activation='softmax', name='auxilliary_output_2')(x2)

x = inception_module(x,256,160,320,32,128,128,name='inception_4e')

x = tf.keras.layers.MaxPooling2D((3,3),strides=(2,2),padding='same',name='max_pool_4_3x3_2')(x)

x = inception_module(x,256,160,320,32,128,128,name = 'inception_5a')

x = inception_module(x,384,192,384,48,128,128,name='inception_5b')

x = tf.keras.layers.AveragePooling2D((7,7),strides=(1,1),name='avg_pool_5_7x7_1')(x)

x = tf.keras.layers.Dropout(0.4)(x)



x = tf.keras.layers.Flatten(name='output_flatten')(x)
x = tf.keras.layers.Dense(197,activation='linear',name='linear')(x)
x = tf.keras.layers.Dense(10,activation='softmax',name='output')(x)

model = tf.keras.Model(input_layer, [x, x1, x2], name='inception_v1')

model.summary()



Model: "inception_v1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_34      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_1_7x7_2        │ (None, 112, 112,  │      9,472 │ input_layer_34[0… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pool_1_3x3_2    │ (None, 56, 56,    │          0 │ conv_1_7x7_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_2b_3x3_1       │ (None, 56, 56,    │    110,784 │ max_pool_1_3x3_2… │
│ (Conv2D)            │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pool_2_3x3_2    │ (None, 28, 28,    │          0 │ conv_2b_3x3_1[0]… │
│ (MaxPooling2D)      │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1587         │ (None, 28, 28,    │     18,528 │ max_pool_2_3x3_2… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1589         │ (None, 28, 28,    │      3,088 │ max_pool_2_3x3_2… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_254   │ (None, 28, 28,    │          0 │ max_pool_2_3x3_2… │
│ (MaxPooling2D)      │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1586         │ (None, 28, 28,    │     12,352 │ max_pool_2_3x3_2… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1588         │ (None, 28, 28,    │    110,720 │ conv2d_1587[0][0] │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1590         │ (None, 28, 28,    │     12,832 │ conv2d_1589[0][0] │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1591         │ (None, 28, 28,    │      6,176 │ max_pooling2d_25… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ inception_3a        │ (None, 28, 28,    │          0 │ conv2d_1586[0][0… │
│ (Concatenate)       │ 256)              │            │ conv2d_1588[0][0… │
│                     │                   │            │ conv2d_1590[0][0… │
│                     │                   │            │ conv2d_1591[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1593         │ (None, 28, 28,    │     32,896 │ inception_3a[0][… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1595         │ (None, 28, 28,    │      8,224 │ inception_3a[0][… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_255   │ (None, 28, 28,    │          0 │ inception_3a[0][… │
│ (MaxPooling2D)      │ 256)              │            │                 

 Total params: 10,523,525 (40.14 MB)

 Trainable params: 10,523,525 (40.14 MB)

 Non-trainable params: 0 (0.00 B)